In [ ]:
pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.3/317.3 MB 1.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.2-py2.py3-none-any.whl size=317812365 sha256=7288aca29bc44e379e32f5501e6b358fabfc42a9b580418784b9d63ab10e1889
  Stored in directory: /root/.cache/pip/wheels/34/34/bd/03944534c44b677cd5859f248090daa9fb27b3c8f8e5f49574
Successfully built pyspark


# Import libraries


In [ ]:
import pyspark
from pyspark.sql import SparkSession
#StructField and StructType are in pyspark.sql.types
from pyspark.sql.types import StructType,StructField, StringType, IntegerType
from pyspark.sql.types import *

# Create DataFrame

In [ ]:
spark = SparkSession.builder.appName('MyApp').getOrCreate()

df = spark.read.load('/content/googleplaystore.csv', format='csv', sep=',', header=True, escape='"', inferSchema=True)

In [ ]:
df.count()

10841

In [ ]:
df.show(1)

+--------------------+--------------+------+-------+----+--------+----+-----+--------------+------------+---------------+-----------+------------+
|                 App|      Category|Rating|Reviews|Size|Installs|Type|Price|Content Rating|      Genres|   Last Updated|Current Ver| Android Ver|
+--------------------+--------------+------+-------+----+--------+----+-----+--------------+------------+---------------+-----------+------------+
|Photo Editor & Ca...|ART_AND_DESIGN|   4.1|    159| 19M| 10,000+|Free|    0|      Everyone|Art & Design|January 7, 2018|      1.0.0|4.0.3 and up|
+--------------------+--------------+------+-------+----+--------+----+-----+--------------+------------+---------------+-----------+------------+
only showing top 1 row



# Check Schema

In [ ]:
df.printSchema()

root
 |-- App: string (nullable = true)
 |-- Category: string (nullable = true)
 |-- Rating: double (nullable = true)
 |-- Reviews: string (nullable = true)
 |-- Size: string (nullable = true)
 |-- Installs: string (nullable = true)
 |-- Type: string (nullable = true)
 |-- Price: string (nullable = true)
 |-- Content Rating: string (nullable = true)
 |-- Genres: string (nullable = true)
 |-- Last Updated: string (nullable = true)
 |-- Current Ver: string (nullable = true)
 |-- Android Ver: string (nullable = true)



# Data Cleaning

In [ ]:
df = df.drop('size', 'Content Rating', 'Last Updated', 'Android Ver')

In [ ]:
df.show()

+--------------------+--------------+------+-------+-----------+----+-----+--------------------+------------------+
|                 App|      Category|Rating|Reviews|   Installs|Type|Price|              Genres|       Current Ver|
+--------------------+--------------+------+-------+-----------+----+-----+--------------------+------------------+
|Photo Editor & Ca...|ART_AND_DESIGN|   4.1|    159|    10,000+|Free|    0|        Art & Design|             1.0.0|
| Coloring book moana|ART_AND_DESIGN|   3.9|    967|   500,000+|Free|    0|Art & Design;Pret...|             2.0.0|
|U Launcher Lite –...|ART_AND_DESIGN|   4.7|  87510| 5,000,000+|Free|    0|        Art & Design|             1.2.4|
|Sketch - Draw & P...|ART_AND_DESIGN|   4.5| 215644|50,000,000+|Free|    0|        Art & Design|Varies with device|
|Pixel Draw - Numb...|ART_AND_DESIGN|   4.3|    967|   100,000+|Free|    0|Art & Design;Crea...|               1.1|
|Paper flowers ins...|ART_AND_DESIGN|   4.4|    167|    50,000+|Free|   

In [ ]:
df = df.drop('Current Ver')

In [ ]:
df.show(2)

+--------------------+--------------+------+-------+--------+----+-----+--------------------+
|                 App|      Category|Rating|Reviews|Installs|Type|Price|              Genres|
+--------------------+--------------+------+-------+--------+----+-----+--------------------+
|Photo Editor & Ca...|ART_AND_DESIGN|   4.1|    159| 10,000+|Free|    0|        Art & Design|
| Coloring book moana|ART_AND_DESIGN|   3.9|    967|500,000+|Free|    0|Art & Design;Pret...|
+--------------------+--------------+------+-------+--------+----+-----+--------------------+
only showing top 2 rows



In [ ]:
df.printSchema()

root
 |-- App: string (nullable = true)
 |-- Category: string (nullable = true)
 |-- Rating: double (nullable = true)
 |-- Reviews: string (nullable = true)
 |-- Installs: string (nullable = true)
 |-- Type: string (nullable = true)
 |-- Price: string (nullable = true)
 |-- Genres: string (nullable = true)



In [ ]:
from pyspark.sql.functions import regexp_replace, col
df = df.withColumn('Reviews', col('Reviews').cast(IntegerType()))\
.withColumn('Installs', regexp_replace('Installs', '[^0-9]', ''))\
.withColumn('Installs', col('Installs').cast(IntegerType()))\
.withColumn('Price', regexp_replace(col('Price'), '[$]', ''))\
.withColumn('Price', col('Price').cast(IntegerType()))


In [ ]:
df.show(2)

+--------------------+--------------+------+-------+--------+----+-----+--------------------+
|                 App|      Category|Rating|Reviews|Installs|Type|Price|              Genres|
+--------------------+--------------+------+-------+--------+----+-----+--------------------+
|Photo Editor & Ca...|ART_AND_DESIGN|   4.1|    159|   10000|Free|    0|        Art & Design|
| Coloring book moana|ART_AND_DESIGN|   3.9|    967|  500000|Free|    0|Art & Design;Pret...|
+--------------------+--------------+------+-------+--------+----+-----+--------------------+
only showing top 2 rows



In [32]:
df.createOrReplaceTempView('apps')
spark.sql("SELECT * FROM apps").show()

+--------------------+--------------+------+-------+--------+----+-----+--------------------+
|                 App|      Category|Rating|Reviews|Installs|Type|Price|              Genres|
+--------------------+--------------+------+-------+--------+----+-----+--------------------+
|Photo Editor & Ca...|ART_AND_DESIGN|   4.1|    159|   10000|Free|    0|        Art & Design|
| Coloring book moana|ART_AND_DESIGN|   3.9|    967|  500000|Free|    0|Art & Design;Pret...|
|U Launcher Lite –...|ART_AND_DESIGN|   4.7|  87510| 5000000|Free|    0|        Art & Design|
|Sketch - Draw & P...|ART_AND_DESIGN|   4.5| 215644|50000000|Free|    0|        Art & Design|
|Pixel Draw - Numb...|ART_AND_DESIGN|   4.3|    967|  100000|Free|    0|Art & Design;Crea...|
|Paper flowers ins...|ART_AND_DESIGN|   4.4|    167|   50000|Free|    0|        Art & Design|
|Smoke Effect Phot...|ART_AND_DESIGN|   3.8|    178|   50000|Free|    0|        Art & Design|
|    Infinite Painter|ART_AND_DESIGN|   4.1|  36815| 1000000

# Most Reviewed Apps

In [35]:
spark.sql("SELECT App, sum(Reviews) from Apps group by App order by sum(Reviews) desc limit 10").show()

+--------------------+------------+
|                 App|sum(Reviews)|
+--------------------+------------+
|           Instagram|   266241989|
|  WhatsApp Messenger|   207348304|
|      Clash of Clans|   179558781|
|Messenger – Text ...|   169932272|
|      Subway Surfers|   166331958|
|    Candy Crush Saga|   156993136|
|            Facebook|   156286514|
|         8 Ball Pool|    99386198|
|        Clash Royale|    92530298|
|            Snapchat|    68045010|
+--------------------+------------+



# Most Installed Apps

In [37]:
spark.sql("SELECT App, Type, sum(Installs) from Apps group by App, Type order by sum(Installs) desc limit 10").show()

+------------------+----+-------------+
|               App|Type|sum(Installs)|
+------------------+----+-------------+
|    Subway Surfers|Free|   6000000000|
|         Instagram|Free|   4000000000|
|      Google Drive|Free|   4000000000|
|          Hangouts|Free|   4000000000|
|     Google Photos|Free|   4000000000|
|       Google News|Free|   4000000000|
|  Candy Crush Saga|Free|   3500000000|
|WhatsApp Messenger|Free|   3000000000|
|             Gmail|Free|   3000000000|
|      Temple Run 2|Free|   3000000000|
+------------------+----+-------------+



# Category Wise Distribution

In [38]:
spark.sql("SELECT Category, sum(Installs) from Apps group by Category order by sum(installs) desc").show()

+-------------------+-------------+
|           Category|sum(Installs)|
+-------------------+-------------+
|               GAME|  35086024415|
|      COMMUNICATION|  32647276251|
|       PRODUCTIVITY|  14176091369|
|             SOCIAL|  14069867902|
|              TOOLS|  11452771915|
|             FAMILY|  10258263505|
|        PHOTOGRAPHY|  10088247655|
| NEWS_AND_MAGAZINES|   7496317760|
|   TRAVEL_AND_LOCAL|   6868887146|
|      VIDEO_PLAYERS|   6222002720|
|           SHOPPING|   3247848785|
|      ENTERTAINMENT|   2869160000|
|    PERSONALIZATION|   2325494782|
|BOOKS_AND_REFERENCE|   1921469576|
|             SPORTS|   1751174498|
| HEALTH_AND_FITNESS|   1583072512|
|           BUSINESS|   1001914865|
|            FINANCE|    876648734|
|          EDUCATION|    871452000|
|MAPS_AND_NAVIGATION|    724281890|
+-------------------+-------------+
only showing top 20 rows



# Top Paid Apps

In [41]:
spark.sql("SELECT App, sum(Price) from Apps where Type = 'Paid' group by App order by sum(Price) desc").show()

+--------------------+----------+
|                 App|sum(Price)|
+--------------------+----------+
|I'm Rich - Trump ...|       400|
|   I Am Rich Premium|       399|
|  I AM RICH PRO PLUS|       399|
|I'm Rich/Eu sou R...|       399|
|      I am Rich Plus|       399|
|most expensive ap...|       399|
|       I Am Rich Pro|       399|
|  I am rich(premium)|       399|
|           I am Rich|       399|
|          I am Rich!|       399|
|         💎 I'm rich|       399|
|I am rich (Most e...|       399|
|           I am rich|       399|
|         Eu Sou Rico|       394|
|           I Am Rich|       389|
| I am extremely Rich|       379|
|       I am rich VIP|       299|
|        EP Cook Book|       200|
|Vargo Anesthesia ...|       158|
|       cronometra-br|       154|
+--------------------+----------+
only showing top 20 rows

